# Q-Learning 基础与实现

---

## 学习目标

通过本教程，你将学会：
- 理解时序差分学习 (TD Learning) 的核心思想
- 掌握 Q-Learning 算法的数学原理
- 实现表格型 Q-Learning
- 理解探索与利用的平衡策略
- 在悬崖行走环境中训练智能体

## 前置知识

- 马尔可夫决策过程 (MDP) 基本概念
- Python 和 NumPy 基础
- 概率论基础

## 预计时间

45-60 分钟

---

## 第1部分：从动态规划到无模型学习

### 1.1 动态规划的局限性

在 MDP 基础模块中，我们学习了动态规划 (DP) 方法求解最优策略。DP 方法需要：

1. **完整的环境模型**：状态转移概率 $P(s'|s,a)$ 和奖励函数 $R(s,a,s')$
2. **遍历所有状态和动作**：计算复杂度随状态空间增大而爆炸

**现实问题**：
- 转移概率通常未知（如何精确计算开车时每个操作的后果？）
- 状态空间可能极其巨大（围棋约有 $10^{170}$ 种状态）

### 1.2 无模型强化学习

**核心思想**：通过与环境**交互采样**学习最优策略，无需事先知道环境模型。

```
┌─────────────────────────────────────────────────────────────┐
│                    强化学习方法分类                          │
├─────────────────────────────────────────────────────────────┤
│                                                             │
│  ┌─────────────┐           ┌─────────────────────────────┐  │
│  │  基于模型   │           │        无模型 (Model-Free)   │  │
│  │ (Model-Based)│          ├──────────────┬──────────────┤  │
│  │             │           │  基于价值    │  基于策略     │  │
│  │ • 动态规划   │           │ (Value-Based)│(Policy-Based)│  │
│  │ • 模型预测控制│          │              │              │  │
│  │ • Dyna-Q    │           │ • Q-Learning │ • REINFORCE  │  │
│  └─────────────┘           │ • SARSA      │ • Actor-Critic│  │
│                            │ • DQN        │ • PPO        │  │
│                            └──────────────┴──────────────┘  │
└─────────────────────────────────────────────────────────────┘
```

---

## 第2部分：时序差分学习 (TD Learning)

### 2.1 蒙特卡洛 vs 时序差分

**蒙特卡洛方法** (Monte Carlo, MC)：

$$V(S_t) \leftarrow V(S_t) + \alpha \left[ G_t - V(S_t) \right]$$

- 需要等待**回合结束**才能更新
- $G_t = R_{t+1} + \gamma R_{t+2} + \gamma^2 R_{t+3} + ...$ 是完整回合的实际累积回报
- 无偏估计，但方差较大

**时序差分方法** (Temporal Difference, TD)：

$$V(S_t) \leftarrow V(S_t) + \alpha \left[ R_{t+1} + \gamma V(S_{t+1}) - V(S_t) \right]$$

- **每一步**都可以更新
- 使用**自举** (Bootstrapping)：用估计值更新估计值
- 有偏估计，但方差较小

### 2.2 TD 误差

**TD 目标** (TD Target):

$$\text{TD Target} = R_{t+1} + \gamma V(S_{t+1})$$

**TD 误差** (TD Error):

$$\delta_t = R_{t+1} + \gamma V(S_{t+1}) - V(S_t)$$

**直觉理解**：
- $\delta_t > 0$：实际比预期好，应增大 $V(S_t)$
- $\delta_t < 0$：实际比预期差，应减小 $V(S_t)$
- $\delta_t = 0$：预测准确，价值已收敛

---

## 第3部分：Q-Learning 算法

### 3.1 算法原理

Q-Learning 是一种**离策略** (Off-Policy) TD 控制算法。它直接学习最优动作价值函数 $Q^*$。

**更新公式**：

$$Q(S_t, A_t) \leftarrow Q(S_t, A_t) + \alpha \left[ R_{t+1} + \gamma \max_{a} Q(S_{t+1}, a) - Q(S_t, A_t) \right]$$

**关键特性**：
- 使用 $\max$ 操作选择下一状态的最大 Q 值
- 与当前实际采取的行为策略无关 (Off-Policy)
- 在一定条件下保证收敛到最优 Q 函数

### 3.2 算法伪代码

```
算法: Q-Learning

输入: 状态空间 S, 动作空间 A, 学习率 α, 折扣因子 γ, 探索率 ε
输出: 最优 Q 函数

1. 初始化 Q(s, a) = 0，对于所有 s ∈ S, a ∈ A
2. 对于每个回合:
   a. 初始化状态 S
   b. 重复 (对于回合中的每一步):
      i.   使用 ε-greedy 从 Q 选择动作 A
      ii.  执行动作 A，观察奖励 R 和下一状态 S'
      iii. Q(S, A) ← Q(S, A) + α[R + γ max_a Q(S', a) - Q(S, A)]
      iv.  S ← S'
   c. 直到 S 是终止状态
3. 返回 Q
```

---

## 第4部分：代码实现

### 步骤1: 导入库和配置

In [ ]:
# ============================================================
# 导入必要的库
# ============================================================

import numpy as np
from collections import defaultdict
from typing import Tuple, List, Dict, Any, Optional
import matplotlib.pyplot as plt

# ============================================================
# 配置参数
# ============================================================

# 设置随机种子，确保结果可重复
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

# 可视化配置
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 11

print("库导入完成")

### 步骤2: 实现悬崖行走环境

悬崖行走 (Cliff Walking) 是经典的强化学习测试环境：

```
┌─────────────────────────────────────────────┐
│ .  .  .  .  .  .  .  .  .  .  .  .  │  row 0
│ .  .  .  .  .  .  .  .  .  .  .  .  │  row 1
│ .  .  .  .  .  .  .  .  .  .  .  .  │  row 2
│ S  C  C  C  C  C  C  C  C  C  C  G  │  row 3
└─────────────────────────────────────────────┘
  0  1  2  3  4  5  6  7  8  9 10 11

S: 起点    G: 目标    C: 悬崖
```

- 动作空间：上(0)、右(1)、下(2)、左(3)
- 奖励：每步 -1，掉入悬崖 -100 并重置到起点

In [ ]:
class CliffWalkingEnv:
    """
    悬崖行走环境
    
    经典的强化学习测试环境，用于对比不同算法的行为特性。
    智能体需要从起点 S 到达目标 G，同时避免掉入悬崖 C。
    """
    
    # 动作定义：(行偏移, 列偏移)
    ACTIONS = {
        0: (-1, 0),   # 上
        1: (0, 1),    # 右
        2: (1, 0),    # 下
        3: (0, -1)    # 左
    }
    ACTION_NAMES = ['上', '右', '下', '左']
    
    def __init__(self, height: int = 4, width: int = 12):
        """
        初始化环境
        
        Args:
            height: 网格高度
            width: 网格宽度
        """
        self.height = height
        self.width = width
        
        # 特殊位置定义
        self.start = (height - 1, 0)           # 起点：左下角
        self.goal = (height - 1, width - 1)    # 终点：右下角
        self.cliff = [(height - 1, j) for j in range(1, width - 1)]  # 悬崖：底部中间
        
        # 当前状态
        self.state = self.start
        self.n_actions = 4
        
    def reset(self) -> Tuple[int, int]:
        """重置环境到初始状态"""
        self.state = self.start
        return self.state
    
    def step(self, action: int) -> Tuple[Tuple[int, int], float, bool]:
        """
        执行动作
        
        Args:
            action: 动作索引 (0-3)
            
        Returns:
            (next_state, reward, done) 三元组
        """
        # 计算下一位置（边界裁剪）
        di, dj = self.ACTIONS[action]
        new_i = np.clip(self.state[0] + di, 0, self.height - 1)
        new_j = np.clip(self.state[1] + dj, 0, self.width - 1)
        next_state = (int(new_i), int(new_j))
        
        # 检查是否掉入悬崖
        if next_state in self.cliff:
            self.state = self.start  # 重置到起点
            return self.state, -100.0, False
        
        self.state = next_state
        
        # 检查是否到达目标
        if self.state == self.goal:
            return self.state, 0.0, True
        
        return self.state, -1.0, False
    
    def render(self, path: Optional[List[Tuple[int, int]]] = None) -> None:
        """可视化环境"""
        grid = [['.' for _ in range(self.width)] for _ in range(self.height)]
        
        # 标记悬崖
        for pos in self.cliff:
            grid[pos[0]][pos[1]] = 'C'
        
        # 标记起点和终点
        grid[self.start[0]][self.start[1]] = 'S'
        grid[self.goal[0]][self.goal[1]] = 'G'
        
        # 标记路径
        if path:
            for pos in path[1:-1]:
                if pos not in self.cliff and pos != self.start and pos != self.goal:
                    grid[pos[0]][pos[1]] = '*'
        
        # 打印网格
        print("┌" + "─" * (self.width * 2 + 1) + "┐")
        for row in grid:
            print("│ " + " ".join(row) + " │")
        print("└" + "─" * (self.width * 2 + 1) + "┘")


# 测试环境
env = CliffWalkingEnv()
print("悬崖行走环境:")
env.render()
print(f"\n起点: {env.start}")
print(f"终点: {env.goal}")
print(f"悬崖位置: {env.cliff[:3]}...{env.cliff[-1]}")

### 步骤3: 实现 Q-Learning 智能体

In [ ]:
class QLearningAgent:
    """
    表格型 Q-Learning 智能体
    
    实现标准的 Q-Learning 算法，使用 ε-greedy 策略进行探索。
    
    Attributes:
        q_table: Q 值表，字典形式 {state: [Q(s,a0), Q(s,a1), ...]}
        lr: 学习率 α
        gamma: 折扣因子 γ
        epsilon: 探索率
    """
    
    def __init__(
        self,
        n_actions: int,
        learning_rate: float = 0.1,
        discount_factor: float = 0.99,
        epsilon: float = 1.0,
        epsilon_decay: float = 0.995,
        epsilon_min: float = 0.01
    ):
        """
        初始化 Q-Learning 智能体
        
        Args:
            n_actions: 动作空间大小
            learning_rate: 学习率，控制 Q 值更新步长
            discount_factor: 折扣因子，权衡即时与未来奖励
            epsilon: 初始探索率
            epsilon_decay: 探索率衰减系数
            epsilon_min: 最小探索率
        """
        self.n_actions = n_actions
        self.lr = learning_rate
        self.gamma = discount_factor
        self.epsilon = epsilon
        self.epsilon_decay = epsilon_decay
        self.epsilon_min = epsilon_min
        
        # Q 表：使用 defaultdict 自动初始化未访问状态
        self.q_table: Dict[Any, np.ndarray] = defaultdict(
            lambda: np.zeros(n_actions)
        )
        
    def get_action(self, state: Any, training: bool = True) -> int:
        """
        使用 ε-greedy 策略选择动作
        
        Args:
            state: 当前状态
            training: 是否处于训练模式
            
        Returns:
            选择的动作索引
        """
        # 训练时以 ε 概率随机探索
        if training and np.random.random() < self.epsilon:
            return np.random.randint(self.n_actions)
        
        # 利用：选择 Q 值最大的动作（打破平局时随机选择）
        q_values = self.q_table[state]
        max_q = np.max(q_values)
        max_actions = np.where(np.isclose(q_values, max_q))[0]
        return np.random.choice(max_actions)
    
    def update(
        self,
        state: Any,
        action: int,
        reward: float,
        next_state: Any,
        done: bool
    ) -> float:
        """
        Q-Learning 更新规则
        
        Q(S,A) ← Q(S,A) + α[R + γ max_a Q(S',a) - Q(S,A)]
        
        Args:
            state: 当前状态
            action: 执行的动作
            reward: 获得的奖励
            next_state: 下一状态
            done: 是否终止
            
        Returns:
            TD 误差
        """
        current_q = self.q_table[state][action]
        
        # 计算 TD 目标
        if done:
            target = reward
        else:
            # Q-Learning 核心：使用 max 选择下一状态的最优动作
            target = reward + self.gamma * np.max(self.q_table[next_state])
        
        # TD 误差
        td_error = target - current_q
        
        # 更新 Q 值
        self.q_table[state][action] += self.lr * td_error
        
        return td_error
    
    def decay_epsilon(self) -> None:
        """衰减探索率"""
        self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)


# 创建智能体实例
agent = QLearningAgent(
    n_actions=4,
    learning_rate=0.5,
    discount_factor=0.99,
    epsilon=0.1,
    epsilon_decay=1.0,  # 保持固定探索率
    epsilon_min=0.1
)

print("Q-Learning 智能体配置:")
print(f"  动作数量: {agent.n_actions}")
print(f"  学习率 α: {agent.lr}")
print(f"  折扣因子 γ: {agent.gamma}")
print(f"  探索率 ε: {agent.epsilon}")

### 步骤4: 训练循环实现

In [ ]:
def train_q_learning(
    env: CliffWalkingEnv,
    agent: QLearningAgent,
    episodes: int = 500,
    max_steps: int = 200,
    verbose: bool = True
) -> Dict[str, List[float]]:
    """
    训练 Q-Learning 智能体
    
    Args:
        env: 环境实例
        agent: Q-Learning 智能体
        episodes: 训练回合数
        max_steps: 每回合最大步数
        verbose: 是否打印训练进度
        
    Returns:
        训练历史记录
    """
    history = {
        'rewards': [],
        'steps': [],
        'epsilon': []
    }
    
    for episode in range(episodes):
        state = env.reset()
        total_reward = 0.0
        steps = 0
        
        for step in range(max_steps):
            # 选择动作
            action = agent.get_action(state, training=True)
            
            # 执行动作
            next_state, reward, done = env.step(action)
            
            # 更新 Q 值
            agent.update(state, action, reward, next_state, done)
            
            state = next_state
            total_reward += reward
            steps += 1
            
            if done:
                break
        
        # 衰减探索率
        agent.decay_epsilon()
        
        # 记录历史
        history['rewards'].append(total_reward)
        history['steps'].append(steps)
        history['epsilon'].append(agent.epsilon)
        
        # 打印进度
        if verbose and (episode + 1) % 100 == 0:
            avg_reward = np.mean(history['rewards'][-100:])
            avg_steps = np.mean(history['steps'][-100:])
            print(f"Episode {episode + 1:4d} | "
                  f"Avg Reward: {avg_reward:8.2f} | "
                  f"Avg Steps: {avg_steps:6.1f} | "
                  f"ε: {agent.epsilon:.4f}")
    
    return history


# 训练智能体
print("开始训练 Q-Learning...\n")
env = CliffWalkingEnv()
agent = QLearningAgent(
    n_actions=4,
    learning_rate=0.5,
    discount_factor=0.99,
    epsilon=0.1,
    epsilon_decay=1.0,
    epsilon_min=0.1
)

history = train_q_learning(env, agent, episodes=500)

print(f"\n训练完成！最后100回合平均奖励: {np.mean(history['rewards'][-100:]):.2f}")

### 步骤5: 可视化学习过程

In [ ]:
def plot_training_history(history: Dict[str, List[float]], window: int = 10) -> None:
    """绘制训练历史曲线"""
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # 奖励曲线
    rewards = history['rewards']
    smoothed_rewards = np.convolve(
        rewards, np.ones(window)/window, mode='valid'
    )
    
    axes[0].plot(rewards, alpha=0.3, color='blue', label='原始奖励')
    axes[0].plot(range(window-1, len(rewards)), smoothed_rewards, 
                 color='blue', linewidth=2, label=f'{window}回合移动平均')
    axes[0].set_xlabel('Episode')
    axes[0].set_ylabel('Total Reward')
    axes[0].set_title('Q-Learning 学习曲线')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # 步数曲线
    steps = history['steps']
    smoothed_steps = np.convolve(
        steps, np.ones(window)/window, mode='valid'
    )
    
    axes[1].plot(steps, alpha=0.3, color='green', label='原始步数')
    axes[1].plot(range(window-1, len(steps)), smoothed_steps,
                 color='green', linewidth=2, label=f'{window}回合移动平均')
    axes[1].set_xlabel('Episode')
    axes[1].set_ylabel('Steps')
    axes[1].set_title('每回合步数')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()


plot_training_history(history)

### 步骤6: 提取并可视化学到的策略

In [ ]:
def extract_path(agent: QLearningAgent, env: CliffWalkingEnv, max_steps: int = 50) -> List[Tuple[int, int]]:
    """从训练好的智能体提取贪心策略路径"""
    state = env.reset()
    path = [state]
    
    for _ in range(max_steps):
        action = agent.get_action(state, training=False)  # 不探索
        next_state, _, done = env.step(action)
        path.append(next_state)
        state = next_state
        if done:
            break
    
    return path


def visualize_policy(agent: QLearningAgent, env: CliffWalkingEnv) -> None:
    """可视化学到的策略"""
    arrow_map = {0: '↑', 1: '→', 2: '↓', 3: '←'}
    
    print("学到的策略 (贪心):")
    print("┌" + "───" * env.width + "┐")
    
    for i in range(env.height):
        row = "│"
        for j in range(env.width):
            state = (i, j)
            if state == env.start:
                row += " S "
            elif state == env.goal:
                row += " G "
            elif state in env.cliff:
                row += " C "
            elif state in agent.q_table:
                best_action = np.argmax(agent.q_table[state])
                row += f" {arrow_map[best_action]} "
            else:
                row += " . "
        print(row + "│")
    
    print("└" + "───" * env.width + "┘")


# 可视化策略
visualize_policy(agent, env)

# 提取并显示路径
print("\n学到的路径:")
path = extract_path(agent, env)
env.reset()
env.render(path)
print(f"路径长度: {len(path) - 1} 步")

### 步骤7: Q 值表可视化

In [ ]:
def visualize_q_table(agent: QLearningAgent, env: CliffWalkingEnv) -> None:
    """可视化 Q 表和价值函数"""
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # 准备价值函数数据
    v_table = np.zeros((env.height, env.width))
    for i in range(env.height):
        for j in range(env.width):
            state = (i, j)
            if state in agent.q_table:
                v_table[i, j] = np.max(agent.q_table[state])
            else:
                v_table[i, j] = 0
    
    # 价值函数热力图
    im = axes[0].imshow(v_table, cmap='RdYlGn', aspect='auto')
    axes[0].set_title('状态价值函数 V(s) = max_a Q(s,a)')
    axes[0].set_xlabel('列')
    axes[0].set_ylabel('行')
    plt.colorbar(im, ax=axes[0])
    
    # 标记悬崖位置
    for pos in env.cliff:
        axes[0].add_patch(plt.Rectangle(
            (pos[1]-0.5, pos[0]-0.5), 1, 1,
            fill=True, color='black', alpha=0.5
        ))
    
    # Q 值分布直方图
    q_values = []
    for state, q_array in agent.q_table.items():
        if state not in env.cliff:
            q_values.extend(q_array.tolist())
    
    axes[1].hist(q_values, bins=50, edgecolor='black', alpha=0.7, color='steelblue')
    axes[1].set_xlabel('Q Value')
    axes[1].set_ylabel('Frequency')
    axes[1].set_title('Q 值分布')
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()


visualize_q_table(agent, env)

---

## 第5部分：探索策略

### 5.1 探索与利用困境

强化学习面临的核心挑战之一：

- **利用 (Exploitation)**：选择当前已知最优的动作，最大化即时收益
- **探索 (Exploration)**：尝试新动作，可能发现更好的策略

**过度利用**：可能陷入局部最优，错过全局最优解
**过度探索**：无法收敛，浪费计算资源

### 5.2 ε-Greedy 策略

最简单的探索策略：
- 以概率 $\epsilon$ 随机选择动作（探索）
- 以概率 $1-\epsilon$ 选择最优动作（利用）

### 5.3 其他探索策略

In [ ]:
# ============================================================
# 不同探索策略实现
# ============================================================

def epsilon_greedy(q_values: np.ndarray, epsilon: float) -> int:
    """
    ε-Greedy 策略
    
    以概率 ε 随机选择，否则选择最优动作
    """
    if np.random.random() < epsilon:
        return np.random.randint(len(q_values))
    return np.argmax(q_values)


def softmax_action(q_values: np.ndarray, temperature: float) -> int:
    """
    Softmax (Boltzmann) 策略
    
    根据 Q 值的 softmax 分布选择动作:
        P(a|s) = exp(Q(s,a)/τ) / Σ exp(Q(s,a')/τ)
    
    温度 τ 控制探索程度:
        - τ→0: 趋向贪心选择
        - τ→∞: 趋向均匀随机
    """
    # 数值稳定性处理
    q_scaled = (q_values - np.max(q_values)) / max(temperature, 1e-8)
    exp_q = np.exp(q_scaled)
    probs = exp_q / np.sum(exp_q)
    return np.random.choice(len(q_values), p=probs)


def ucb_action(q_values: np.ndarray, action_counts: np.ndarray, 
               total_count: int, c: float = 2.0) -> int:
    """
    Upper Confidence Bound (UCB) 策略
    
    选择置信上界最大的动作:
        A = argmax[Q(s,a) + c * sqrt(ln(t) / N(s,a))]
    
    平衡价值估计和不确定性
    """
    # 优先选择未访问的动作
    if np.any(action_counts == 0):
        return np.random.choice(np.where(action_counts == 0)[0])
    
    ucb_values = q_values + c * np.sqrt(np.log(total_count + 1) / (action_counts + 1e-8))
    return np.argmax(ucb_values)


# 演示不同策略的行为
print("不同探索策略的动作选择演示")
print("="*50)

q_values = np.array([1.0, 2.0, 1.5, 0.5])  # 假设的 Q 值
action_counts = np.array([10, 5, 8, 2])    # 动作选择计数

print(f"Q 值: {q_values}")
print(f"动作计数: {action_counts}")
print()

# 多次采样观察分布
n_samples = 1000

# ε-Greedy
eg_actions = [epsilon_greedy(q_values, 0.1) for _ in range(n_samples)]
eg_dist = [eg_actions.count(i)/n_samples for i in range(4)]
print(f"ε-Greedy (ε=0.1) 动作分布: {[f'{d:.2f}' for d in eg_dist]}")

# Softmax
sm_actions = [softmax_action(q_values, 0.5) for _ in range(n_samples)]
sm_dist = [sm_actions.count(i)/n_samples for i in range(4)]
print(f"Softmax (τ=0.5) 动作分布: {[f'{d:.2f}' for d in sm_dist]}")

# UCB
ucb_actions = [ucb_action(q_values, action_counts, 100, c=2.0) for _ in range(n_samples)]
ucb_dist = [ucb_actions.count(i)/n_samples for i in range(4)]
print(f"UCB (c=2.0) 动作分布: {[f'{d:.2f}' for d in ucb_dist]}")

---

## 总结

### 核心要点

1. **Q-Learning** 是一种无模型、离策略的时序差分控制算法
2. **更新公式**：$Q(S,A) \leftarrow Q(S,A) + \alpha[R + \gamma \max_a Q(S',a) - Q(S,A)]$
3. **离策略特性**：学习最优策略，与实际行为策略无关
4. **探索与利用**：使用 ε-greedy 等策略平衡

### 超参数选择建议

| 参数 | 典型值 | 说明 |
|------|--------|------|
| $\alpha$ (学习率) | 0.1 ~ 0.5 | 较大值加速学习，但可能不稳定 |
| $\gamma$ (折扣因子) | 0.99 | 接近 1 重视长期奖励 |
| $\epsilon$ (探索率) | 1.0 → 0.01 | 从探索逐渐转向利用 |
| 衰减率 | 0.99 ~ 0.999 | 控制 ε 下降速度 |

### 局限性

- 状态空间必须离散且有限
- 无法处理连续状态（如图像输入）
- 无法泛化到未见过的状态

**解决方案**：深度 Q 网络 (DQN) — 用神经网络近似 Q 函数

### 下一步学习

- SARSA：在策略 TD 控制
- Double Q-Learning：减少过估计
- DQN：深度强化学习入门

---

## 单元测试

运行以下测试验证实现的正确性：

In [ ]:
# ============================================================
# 单元测试
# ============================================================

def run_tests():
    """运行所有单元测试"""
    print("开始单元测试...\n")
    passed = 0
    failed = 0
    
    # 测试1: 环境基本功能
    try:
        env = CliffWalkingEnv()
        state = env.reset()
        assert state == (3, 0), f"起始状态错误: {state}"
        
        # 向上移动
        next_state, reward, done = env.step(0)
        assert next_state == (2, 0), f"向上移动后状态错误: {next_state}"
        assert reward == -1.0, f"奖励错误: {reward}"
        assert not done, "不应该结束"
        
        print("测试1通过: 环境基本功能正常")
        passed += 1
    except AssertionError as e:
        print(f"测试1失败: {e}")
        failed += 1
    
    # 测试2: 悬崖惩罚
    try:
        env = CliffWalkingEnv()
        env.reset()
        # 向右走入悬崖
        next_state, reward, done = env.step(1)
        assert reward == -100.0, f"悬崖惩罚错误: {reward}"
        assert next_state == env.start, f"掉入悬崖后应重置到起点"
        print("测试2通过: 悬崖惩罚正确")
        passed += 1
    except AssertionError as e:
        print(f"测试2失败: {e}")
        failed += 1
    
    # 测试3: Q-Learning 更新
    try:
        agent = QLearningAgent(n_actions=4, learning_rate=0.5, discount_factor=0.9)
        state = (0, 0)
        next_state = (0, 1)
        
        # 初始 Q 值应为 0
        assert agent.q_table[state][0] == 0.0, "初始 Q 值应为 0"
        
        # 执行更新
        td_error = agent.update(state, 0, -1.0, next_state, False)
        
        # 验证更新后的 Q 值
        # Q(s,a) = 0 + 0.5 * (-1 + 0.9 * 0 - 0) = -0.5
        expected_q = -0.5
        assert np.isclose(agent.q_table[state][0], expected_q), \
            f"Q值更新错误: {agent.q_table[state][0]} != {expected_q}"
        
        print("测试3通过: Q-Learning 更新正确")
        passed += 1
    except AssertionError as e:
        print(f"测试3失败: {e}")
        failed += 1
    
    # 测试4: ε-greedy 策略
    try:
        agent = QLearningAgent(n_actions=4, epsilon=0.0)  # 纯贪心
        state = (0, 0)
        agent.q_table[state] = np.array([1.0, 2.0, 0.5, 0.5])
        
        # 应该总是选择动作 1（最大 Q 值）
        for _ in range(10):
            action = agent.get_action(state, training=True)
            assert action == 1, f"贪心策略应选择动作1，实际选择: {action}"
        
        print("测试4通过: ε-greedy 策略正确")
        passed += 1
    except AssertionError as e:
        print(f"测试4失败: {e}")
        failed += 1
    
    # 测试5: 训练收敛性
    try:
        env = CliffWalkingEnv()
        agent = QLearningAgent(
            n_actions=4,
            learning_rate=0.5,
            epsilon=0.1,
            epsilon_decay=1.0,
            epsilon_min=0.1
        )
        
        history = train_q_learning(env, agent, episodes=200, verbose=False)
        
        # 最后50回合平均奖励应该大于 -50
        avg_reward = np.mean(history['rewards'][-50:])
        assert avg_reward > -100, f"训练未收敛: avg_reward = {avg_reward}"
        
        print(f"测试5通过: 训练收敛 (最后50回合平均奖励: {avg_reward:.2f})")
        passed += 1
    except AssertionError as e:
        print(f"测试5失败: {e}")
        failed += 1
    
    # 总结
    print(f"\n{'='*50}")
    print(f"测试完成: {passed} 通过, {failed} 失败")
    if failed == 0:
        print("所有测试通过！")
    else:
        print("存在失败的测试，请检查代码。")
    print(f"{'='*50}")
    
    return failed == 0


# 运行测试
run_tests()

---

## 参考资料

1. Watkins, C.J.C.H. (1989). Learning from Delayed Rewards. PhD Thesis.
2. Sutton, R.S. & Barto, A.G. (2018). Reinforcement Learning: An Introduction, 2nd ed. Chapter 6.
3. [OpenAI Spinning Up - Q-Learning](https://spinningup.openai.com/)

---

[返回目录](../README.md)